# Schedule Recommender System - Baseline Model

This notebook contains baseline models for:
1. Schedule Recommender System
2. A/B Testing Framework

## Setup and Imports


In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy.orm import Session
from sqlalchemy import func, and_
from datetime import datetime
import sys
import os

# Add parent directory to path for imports
sys.path.append('/notebook')

from Database.database import get_db_session, engine
from Database.models import (
    Student, Course, Section, Takes, Prerequisites, 
    TimeSlot, Program, HasCourse, RecommendationResult, ABTestAssignment
)
from Database.db_helpers import (
    get_by_id, get_all, create_record, update_record, 
    delete_record, exists, count_records
)


## Database Connection and Data Loading


In [2]:
# Get database session
db = get_db_session()

# Load data into DataFrames for analysis
def load_data():
    """Load all necessary data from database into pandas DataFrames"""
    
    students_df = pd.read_sql_table('students', engine)
    courses_df = pd.read_sql_table('courses', engine)
    sections_df = pd.read_sql_table('sections', engine)
    takes_df = pd.read_sql_table('takes', engine)
    prerequisites_df = pd.read_sql_table('prerequisites', engine)
    programs_df = pd.read_sql_table('programs', engine)
    hascourse_df = pd.read_sql_table('hascourse', engine)
    timeslots_df = pd.read_sql_table('time_slots', engine)
    
    return {
        'students': students_df,
        'courses': courses_df,
        'sections': sections_df,
        'takes': takes_df,
        'prerequisites': prerequisites_df,
        'programs': programs_df,
        'hascourse': hascourse_df,
        'timeslots': timeslots_df
    }

data = load_data()
print("Data loaded successfully!")
print(f"Students: {len(data['students'])}")
print(f"Courses: {len(data['courses'])}")
print(f"Sections: {len(data['sections'])}")


Data loaded successfully!
Students: 10
Courses: 51
Sections: 51


## Baseline Recommender System

Simple baseline model that recommends courses based on:
1. Program requirements (courses in student's program)
2. Prerequisites (courses student has completed)
3. Popular courses (courses with most enrollments)


In [3]:
class BaselineRecommender:
    """Baseline recommender system for course schedules"""
    
    def __init__(self, data_dict):
        self.data = data_dict
        self._build_features()
    
    def _build_features(self):
        """Build feature matrices for recommendations"""
        # Calculate course popularity (number of enrollments)
        self.course_popularity = self.data['takes'].groupby('section_id').size()
        
        # Map sections to courses
        section_course_map = self.data['sections'].set_index('id')['course_id'].to_dict()
        
        # Calculate course-level popularity
        course_enrollments = {}
        for section_id, count in self.course_popularity.items():
            course_id = section_course_map.get(section_id)
            if course_id:
                course_enrollments[course_id] = course_enrollments.get(course_id, 0) + count
        
        self.course_popularity = pd.Series(course_enrollments)
    
    def get_student_completed_courses(self, student_id):
        """Get courses that student has completed"""
        student_takes = self.data['takes'][
            (self.data['takes']['student_id'] == student_id) & 
            (self.data['takes']['status'] == 'completed')
        ]
        
        if len(student_takes) == 0:
            return set()
        
        # Get course IDs from sections
        section_ids = student_takes['section_id'].values
        completed_courses = self.data['sections'][
            self.data['sections']['id'].isin(section_ids)
        ]['course_id'].unique()
        
        return set(completed_courses)
    
    def get_student_program_courses(self, student_id):
        """Get courses required by student's program"""
        # Get student's program
        student_program = self.data['programs'][
            self.data['programs']['student_id'] == student_id
        ]
        
        if len(student_program) == 0:
            return set()
        
        prog_name = student_program.iloc[0]['prog_name']
        
        # Get courses in program
        program_courses = self.data['hascourse'][
            self.data['hascourse']['prog_name'] == prog_name
        ]['courseid'].unique()
        
        return set(program_courses)
    
    def get_eligible_courses(self, student_id):
        """Get courses student is eligible to take (prerequisites satisfied)"""
        completed = self.get_student_completed_courses(student_id)
        
        # Get all courses
        all_courses = set(self.data['courses']['id'].values)
        
        # Filter courses where prerequisites are satisfied
        eligible = []
        for course_id in all_courses:
            # Get prerequisites for this course
            prereqs = self.data['prerequisites'][
                self.data['prerequisites']['course_id'] == course_id
            ]['prerequisite_id'].values
            
            # If no prerequisites or all prerequisites completed
            if len(prereqs) == 0 or all(prereq in completed for prereq in prereqs):
                eligible.append(course_id)
        
        return set(eligible)
    
    def recommend(self, student_id, n_recommendations=5):
        """Generate recommendations for a student"""
        # Get eligible courses
        eligible = self.get_eligible_courses(student_id)
        
        # Get program courses
        program_courses = self.get_student_program_courses(student_id)
        
        # Get already enrolled courses
        enrolled_sections = self.data['takes'][
            (self.data['takes']['student_id'] == student_id) & 
            (self.data['takes']['status'] == 'enrolled')
        ]['section_id'].values
        
        enrolled_courses = self.data['sections'][
            self.data['sections']['id'].isin(enrolled_sections)
        ]['course_id'].unique()
        enrolled_courses = set(enrolled_courses)
        
        # Filter: eligible, in program, not already enrolled
        candidate_courses = eligible & program_courses - enrolled_courses
        
        if len(candidate_courses) == 0:
            # Fallback: just eligible courses not enrolled
            candidate_courses = eligible - enrolled_courses
        
        # Score courses by popularity
        course_scores = []
        for course_id in candidate_courses:
            popularity = self.course_popularity.get(course_id, 0)
            # Bonus for program courses
            score = popularity + (10 if course_id in program_courses else 0)
            course_scores.append((course_id, score))
        
        # Sort by score and get top N
        course_scores.sort(key=lambda x: x[1], reverse=True)
        top_courses = [course_id for course_id, _ in course_scores[:n_recommendations]]
        
        # Get sections for recommended courses
        recommendations = []
        for course_id in top_courses:
            sections = self.data['sections'][
                self.data['sections']['course_id'] == course_id
            ]
            
            if len(sections) > 0:
                # Get the first available section (can be improved)
                section = sections.iloc[0]
                score = dict(course_scores).get(course_id, 0)
                recommendations.append({
                    'course_id': course_id,
                    'section_id': section['id'],
                    'score': score
                })
        
        return recommendations

# Initialize recommender
recommender = BaselineRecommender(data)
print("Baseline Recommender initialized!")


Baseline Recommender initialized!


# Recommendation Database

In [ ]:
def save_recommendations_to_db(db: Session, student_id: int, recommendations: list, model_version: str = 'baseline_v1'):
    """
    Save recommendations to database using CRUD helpers.
    
    IMPORTANT: The 'recommendation_results' table must exist in the database first.
    Run the "Create Database Tables for Results" cell below to create the table.
    """
    # Convert to Python int in case it's numpy.int64 (from pandas)
    student_id = int(student_id)
    timestamp = datetime.now().isoformat()
    
    # Use CRUD helper to create records
    for rec in recommendations:
        result_data = {
            'student_id': student_id,
            'recommended_section_id': int(rec['section_id']),  # Convert numpy types
            'recommendation_score': str(rec['score']),
            'model_version': model_version,
            'created_at': timestamp
        }
        create_record(db, RecommendationResult, result_data)
    
    print(f"Saved {len(recommendations)} recommendations for student {student_id}")

# Example: Generate recommendations for a student
if len(data['students']) > 0:
    # Convert numpy.int64 to Python int (pandas returns numpy types)
    test_student_id = int(data['students'].iloc[0]['student_id'])
    recommendations = recommender.recommend(test_student_id, n_recommendations=5)
    
    print(f"\nRecommendations for student {test_student_id}:")
    for rec in recommendations:
        course_name = data['courses'][data['courses']['id'] == rec['course_id']].iloc[0]['name']
        print(f"  - {course_name} (Section {rec['section_id']}, Score: {rec['score']})")
    
    # To save recommendations to database:
    # 1. First, run the "Create Database Tables for Results" cell below
    # 2. Then uncomment the line below:
    # save_recommendations_to_db(db, test_student_id, recommendations, 'baseline_v1')


## A/B Testing Framework (Currently fails because 
data is not saved in tables)

Simple A/B testing framework to assign students to different model versions.

**Note:** Uses CRUD helpers for database operations.


In [ ]:
class ABTestFramework:
    """A/B Testing framework for comparing recommendation models"""
    
    def __init__(self, db: Session):
        self.db = db
    
    def assign_student_to_group(self, student_id: int, model_version_a: str = 'baseline_v1', 
                                model_version_b: str = 'baseline_v2') -> str:
        """Assign student to A or B group (50/50 split) - Uses CRUD helpers"""
        # Convert to Python int in case it's numpy.int64 (from pandas)
        student_id = int(student_id)
        
        # Use CRUD helper to check if already assigned
        existing = get_by_id(self.db, ABTestAssignment, student_id=student_id)
        
        if existing:
            return existing.test_group
        
        # Simple 50/50 split based on student_id (deterministic)
        test_group = 'A' if student_id % 2 == 0 else 'B'
        
        # Use CRUD helper to create assignment
        assignment_data = {
            'student_id': student_id,
            'test_group': test_group,
            'model_version_a': model_version_a,
            'model_version_b': model_version_b,
            'assigned_at': datetime.now().isoformat()
        }
        create_record(self.db, ABTestAssignment, assignment_data)
        
        return test_group
    
    def get_student_model_version(self, student_id: int) -> str:
        """Get the model version assigned to a student - Uses CRUD helpers"""
        # Convert to Python int in case it's numpy.int64 (from pandas)
        student_id = int(student_id)
        
        # Use CRUD helper to get assignment
        assignment = get_by_id(self.db, ABTestAssignment, student_id=student_id)
        
        if not assignment:
            # Assign if not already assigned
            group = self.assign_student_to_group(student_id)
            assignment = get_by_id(self.db, ABTestAssignment, student_id=student_id)
        
        if assignment.test_group == 'A':
            return assignment.model_version_a
        else:
            return assignment.model_version_b
    
    def get_test_statistics(self):
        """Get statistics about A/B test assignments"""
        assignments = pd.read_sql_table('ab_test_assignments', engine)
        
        if len(assignments) == 0:
            return {
                'total_students': 0,
                'group_a_count': 0,
                'group_b_count': 0
            }
        
        return {
            'total_students': len(assignments),
            'group_a_count': len(assignments[assignments['test_group'] == 'A']),
            'group_b_count': len(assignments[assignments['test_group'] == 'B'])
        }

# Initialize A/B testing framework
ab_test = ABTestFramework(db)
print("A/B Testing Framework initialized!")

# Example: Assign a student to a test group
if len(data['students']) > 0:
    # Convert numpy.int64 to Python int (pandas returns numpy types)
    test_student_id = int(data['students'].iloc[0]['student_id'])
    group = ab_test.assign_student_to_group(test_student_id)
    model_version = ab_test.get_student_model_version(test_student_id)
    print(f"\nStudent {test_student_id} assigned to group {group}, using model {model_version}")
    
    stats = ab_test.get_test_statistics()
    print(f"\nA/B Test Statistics:")
    print(f"  Total students: {stats['total_students']}")
    print(f"  Group A: {stats['group_a_count']}")
    print(f"  Group B: {stats['group_b_count']}")


A/B Testing Framework initialized!


ProgrammingError: (psycopg2.errors.UndefinedTable) relation "ab_test_assignments" does not exist
LINE 2: FROM ab_test_assignments 
             ^

[SQL: SELECT ab_test_assignments.id AS ab_test_assignments_id, ab_test_assignments.student_id AS ab_test_assignments_student_id, ab_test_assignments.test_group AS ab_test_assignments_test_group, ab_test_assignments.model_version_a AS ab_test_assignments_model_version_a, ab_test_assignments.model_version_b AS ab_test_assignments_model_version_b, ab_test_assignments.assigned_at AS ab_test_assignments_assigned_at 
FROM ab_test_assignments 
WHERE ab_test_assignments.student_id = %(student_id_1)s 
 LIMIT %(param_1)s]
[parameters: {'student_id_1': 1, 'param_1': 1}]
(Background on this error at: https://sqlalche.me/e/20/f405)

## Create Database Tables for Results

**IMPORTANT:** Run this cell FIRST before trying to save recommendations or A/B test assignments to the database.

This creates the following tables:
- `recommendation_results`: Stores recommendation outputs
- `ab_test_assignments`: Stores A/B test group assignments


In [ ]:
from Database.models import Base

# Create tables for recommendation results and A/B test assignments
Base.metadata.create_all(bind=engine)
print("Database tables created successfully!")
